In [1]:
## CSC575 Diana Bursac PreProcessing_Step1

In [2]:
import json
import pandas as pd
import gzip

In [3]:
# LOAD Data INTO Data Frame

In [4]:
import json
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)
    
    
import pandas as pd
import gzip


def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df_music = getDF('Digital_Music_5.json.gz')

In [5]:
df_music.head()

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,5.0,3,True,"06 3, 2013",A2TYZ821XXK2YZ,3426958910,{'Format:': ' Audio CD'},Garrett,"This is awesome to listen to, A must-have for ...",Slayer Rules!,1370217600,NaN
1,5.0,NaN,True,"10 11, 2014",A3OFSREZADFUDY,3426958910,{'Format:': ' Audio CD'},Ad,bien,Five Stars,1412985600,NaN
2,5.0,NaN,True,"02 11, 2014",A2VAMODP8M77NG,3426958910,{'Format:': ' Audio CD'},JTGabq,It was great to hear the old stuff again and I...,SLAYER!!!!!!!!!!!!!!!!!!!!!,1392076800,NaN
3,4.0,3,False,"12 7, 2013",AAKSLZ9IDTEH0,3426958910,{'Format:': ' Audio CD'},john F&#039;n doe,well best of's are a bit poison normally but t...,slayer greatest hits! you mean everything righ...,1386374400,NaN
4,5.0,NaN,True,"06 12, 2016",A3OH43OZJLKI09,5557706259,{'Format:': ' Audio CD'},melinda a goodman,What can I say? This is Casting Crowns!!!This ...,"This is a good, blessing filled",1465689600,NaN


In [6]:
df_music.isnull().sum()

overall                0
vote              162170
verified               0
reviewTime             0
reviewerID             0
asin                   0
style              11792
reviewerName           5
reviewText           158
summary               36
unixReviewTime         0
image             169599
dtype: int64

In [7]:
df_music.shape

(169781, 12)

In [8]:
music = df_music[['overall', 'reviewerID', 'asin', 'reviewText','summary']].copy()
music.rename(columns = {'overall':'rating', 'reviewerID':'userID','asin':'songID'}, inplace = True)

In [9]:
music.head()

,rating,userID,songID,reviewText,summary
0,5.0,A2TYZ821XXK2YZ,3426958910,"This is awesome to listen to, A must-have for ...",Slayer Rules!
1,5.0,A3OFSREZADFUDY,3426958910,bien,Five Stars
2,5.0,A2VAMODP8M77NG,3426958910,It was great to hear the old stuff again and I...,SLAYER!!!!!!!!!!!!!!!!!!!!!
3,4.0,AAKSLZ9IDTEH0,3426958910,well best of's are a bit poison normally but t...,slayer greatest hits! you mean everything righ...
4,5.0,A3OH43OZJLKI09,5557706259,What can I say? This is Casting Crowns!!!This ...,"This is a good, blessing filled"


In [10]:
music.shape

(169781, 5)

In [11]:
music_duplicated = music[music.duplicated(subset = ["userID", 'songID'])]

In [12]:
music_duplicated.shape

(24489, 5)

In [13]:
music_duplicated_R = music[music.duplicated(subset = ["userID", 'songID', 'rating'])]

In [14]:
music_duplicated_R.shape

(24367, 5)

In [15]:
music.drop_duplicates(subset = ["userID", 'songID'], inplace = True)

In [16]:
music.shape

(145292, 5)

In [17]:
# top users are users who rated the most songs, 
# and the top songs are the songs who got the most ratings
# how manu users and songs we want to keep, we have to experiment
# recommendation for a single user: calculate weights btw that user and other users

In [18]:
music.userID.value_counts()

A3W4D8XOGLWUN5    557
A2H3JURQZOHVMB    272
A36EDWL4F3AASU    178
AVKHYC2D7MD56     177
A165P3MOJV3OVZ    176
                 ... 
A2AVMEB8PQOTTE      1
A518S7KT78N51       1
AW9TSXEBXYD5E       1
A70WPHE33U5B        1
A25HR6FTWBQ2NZ      1
Name: userID, Length: 16566, dtype: int64

In [19]:
music.songID.value_counts()

B00CZF8B68    572
B00BWGHIHY    311
B00EH49FRE    277
B0170K9UTO    259
B002HP8EKE    253
             ... 
B00138EYKK      1
B0011UKT0Y      1
B001HDXBZA      1
B00119YKIC      1
B00122YZO2      1
Name: songID, Length: 11797, dtype: int64

In [20]:
M = len(music.songID.unique())

In [21]:
N = len(music.userID.unique())

In [22]:
dict_userID = {uID: count for uID, count in zip(music.userID.unique(),range(N))}
music['userID2'] = music['userID'].apply(lambda x: dict_userID[x])

In [23]:
dict_songsID = {sID: count  for sID, count in zip(music.songID.unique(),range(M))}
music['songID2'] = music['songID'].apply(lambda x: dict_songsID[x])

In [24]:
music.head()

,rating,userID,songID,reviewText,summary,userID2,songID2
0,5.0,A2TYZ821XXK2YZ,3426958910,"This is awesome to listen to, A must-have for ...",Slayer Rules!,0,0
1,5.0,A3OFSREZADFUDY,3426958910,bien,Five Stars,1,0
2,5.0,A2VAMODP8M77NG,3426958910,It was great to hear the old stuff again and I...,SLAYER!!!!!!!!!!!!!!!!!!!!!,2,0
3,4.0,AAKSLZ9IDTEH0,3426958910,well best of's are a bit poison normally but t...,slayer greatest hits! you mean everything righ...,3,0
4,5.0,A3OH43OZJLKI09,5557706259,What can I say? This is Casting Crowns!!!This ...,"This is a good, blessing filled",4,1


In [25]:
music.to_csv('music_edit.csv')